In [1]:
# Train the Vit using Food101 dataset 

In [1]:
%%capture 
%run config.ipynb
%run data-install.ipynb
%run ViT.ipynb

In [2]:
from torch.optim import Adam 
from datetime import datetime 
import torch 
from tqdm import tqdm

In [3]:
model = ViT(
    image_size=IMG_SIZE,
    patch_size=PATCH_SIZE,
    in_channels=IN_CHANNELS,
    n_head=N_HEAD, 
    d_model=D_MODEL, 
    ffn_hidden=FFN_HIDDEN, 
    mlp_hidden=MLP_HIDDEN, 
    n_layers=N_LAYERS, 
    class_num=CLASS_NUM, 
    device=device, 
    drop_prob=DROP_PROB,
)

model.train()

def count_parameters(model): 
    return sum(p.numel() for p in model.parameters() if p.requires_grad) 

logger.info(f'model parameter #: {count_parameters(model)}')

In [4]:
# Setup optimizer 
optimizer = Adam(params = model.parameters(), lr=INIT_LR, decay=WEIGHT_DECAY)

# Setup loss function for training 
loss_func = nn.CrossEntropyLoss()

In [5]:
train_loss_per_epoch = []
test_loss_per_epoch = []

In [6]:
def train_epoch(epoch_num): 
    # Prepare recording CUDA memory snapshot
    # torch.cuda.memory._record_memory_history(
    #     max_entries=100000
    # )
    model.train()
    train_epoch_loss = 0 

    for step, (img, food) in tqdm(enumerate(train_dataloader)):
        optimizer.zero_grad()

        img = img.to(device)
        food = food.to(device)
        out, _ = model(img)

        loss = loss_func(out, food)
        
        loss.backward()

        optimizer.step()

        train_epoch_loss += loss.item()

        if step % 200 == 0: 
            logger.info(f'EPOCH #{epoch_num} STEP #{step} | loss: {loss.item()}, avg_loss: {train_epoch_loss / (step + 1)}')

        # if step == 100:
        #     try:
        #         memory_dump_path = log_dir / f'memory_dump_{timestamp}.log'
        #         torch.cuda.memory._dump_snapshot(memory_dump_path)
        #         # stop recording memory history
        #         torch.cuda.memory._record_memory_history(enabled=None)
        #     except Exception as e:
        #         logger.error(f'Failed to capture memory snapshot {e}')
            

    train_step_loss = train_epoch_loss / (step + 1) 
    return train_epoch_loss, train_step_loss

In [7]:
def evaluate():
    model.eval()
    test_epoch_loss = 0

    with torch.no_grad():
        for step, (img, food) in tqdm(enumerate(test_dataloader)):
            img = img.to(device)
            food = food.to(device)
            out, _ = model(img)

            loss = loss_func(out, food)

            test_epoch_loss += loss.item()

    test_step_loss = test_epoch_loss / (step + 1)

    return test_epoch_loss, test_step_loss

In [ ]:
# Actual training is done here

for epoch in range(EPOCHS):
    train_epoch_loss, train_step_loss = train_epoch(epoch)
    test_epoch_loss, test_step_loss = evaluate()

    train_loss_per_epoch.append(train_step_loss)
    test_loss_per_epoch.append(test_step_loss)

    logger.info(f'Epoch #{epoch} End | Train Loss: {train_step_loss} | Test Loss: {test_step_loss}')

    model_path = model_dir / f'model_{timestamp}_{epoch}'
    torch.save(model.state_dict(), model_path)

logger.info(f'Training Completely Ended!!')


0it [00:00, ?it/s]